<a href="https://colab.research.google.com/github/mille055/AIPI540_individual_project/blob/main/notebooks/AlterDicoms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
## moved a study to the '/content/' folder

!pip install pydicom fastai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import pydicom
import numpy as np
import pandas as pd
import os
from pathlib import Path
from pydicom.dataset import Dataset as DcmDataset
from pydicom.tag import BaseTag as DcmTag
from pydicom.multival import MultiValue as DcmMultiValue
from pydicom.datadict import keyword_for_tag
import os, sys, glob, re
import numpy as np, pandas as pd
from joblib import dump, load
from fastai.basics import delegates
from fastcore.parallel import parallel
from fastai.basics import *

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [38]:
DATA_DIR = 'content/gdrive/MyDrive/WW_MRI_abd2/split/test/028'

In [33]:
def dcmread(fn: Path, no_pixels=True, force=True):
    return pydicom.dcmread(str(fn), stop_before_pixels=no_pixels, force=force)

def cast_dicom_special(x):
    cls = type(x)
    if not cls.__module__.startswith('pydicom'): return x
    if cls.__base__ == object: return x
    return cls.__base__(x)

def split_elem(res, k, v):
    if not isinstance(v, DcmMultiValue): return
    for i, o in enumerate(v): res[f'{k}{"" if i == 0 else i}'] = o


def as_dict(self: DcmDataset, filt=True, split_multi=False):
    if filt:
        vals = [self[o] for o in self.keys() if self[o].keyword in column_lists['dicom_cols']]
        
    else:
        vals = [self[o] for o in self.keys()]
    items = [(v.keyword, v.value.name) if v.keyword == 'SOPClassUID' else (v.keyword, v.value) for v in vals]
    res = dict(items)
    res['fname'] = self.filename
    if split_multi:
        for k, v in items: split_elem(res, k, v)
        for k in res: res[k] = cast_dicom_special(res[k])
    return res
DcmDataset.as_dict = as_dict

# def _dcm2dict(fn, excl_private=False, **kwargs):
#     ds = fn.dcmread(**kwargs)
#     if excl_private: ds.remove_private_tags()
#     return ds.as_dict(**kwargs)

def dcm2dict(fn, excl_private=False, **kwargs):
    ds = dcmread(fn, **kwargs)
    if excl_private: ds.remove_private_tags()
    return ds.as_dict(**kwargs)

# def dcm2dict2(dcm_file, excl_private = False, **kwargs):
#     dcm_data = dcmread(dcm_file, **kwargs)
#     if excl_private: dcm_data.remove_private_tags()
#     dcm_dict = {keyword_for_tag(tag): dcm_data.get(tag) for tag in dcm_data.keys()}
#     return dcm_dict



@delegates(parallel)
def from_dicoms(cls, fns, n_workers=0, **kwargs):
    return pd.DataFrame(parallel(dcm2dict, fns, n_workers=n_workers, **kwargs))
pd.DataFrame.from_dicoms = classmethod(from_dicoms)

In [34]:
def get_dicoms_all_extensions(path, first_dcm=False, **kwargs):
    "Walk `path` to get DICOM file names, then read files into a `pandas.DataFrame`. If `first_dcm=True`, only read first file from each folder."
    
    extension_list = ['.dcm','.dicom', '.dcim', '.ima']
    fns = L()
    print("Finding DICOM files. This may take a few minutes.")
    print('just seeing if this actually updates...')
    if first_dcm:
        for r, d, f in os.walk(path):
                if f:
                    if Path(f[0]).suffix.lower() in extension_list:
                        fns.append(Path(f'{r}/{f[0]}'))
                    else: print(Path(f[0]).suffix.lower(), Path(f[0]), 'is not in extension list')
    else:
        fns = L()
        for r,d,fs in os.walk(path):
            for f in fs:
                if Path(f).suffix.lower() in extension_list: 
                    fns.append(Path(f'{r}/{f}'))
        
    print("Reading DICOMs. This may take a few minutes, depending on the number of files to read...")
    df = pd.DataFrame.from_dicoms(fns, **kwargs)
    return fns, df

In [39]:
dicom_list, dicom_df = get_dicoms_all_extensions(DATA_DIR)

Finding DICOM files. This may take a few minutes.
just seeing if this actually updates...
Reading DICOMs. This may take a few minutes, depending on the number of files to read...


In [41]:
dicom_df

""
